In [1]:
# NEEDED PACKAGES -> no visualization!!

using StochasticDelayDiffEq
using SpecialFunctions
using Distributions
using SimulatedAnnealingABC
using Distances
using DataFrames
using FFTW
using CSV
using XLSX
using ThreadPinning

In [2]:
include("./SDDESolarDynamo.jl")
using .SDDESolarDynamo

In [3]:
# Distance function in the sABC algorithm
function f_dist(θ::Vector{Float64}; type::Int64 = 1, indices::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120, fourier_data::Vector{Float64})
  sol = bfield(θ, Tsim)
  
  simulated_data = sol[1,:]
  simulated_data = simulated_data .^ 2
  fourier_stats = reduced_fourier_spectrum(simulated_data, indices)

  rho = [euclidean(fourier_stats[i], fourier_data[i]) for i in 1:length(fourier_stats)]
  return rho
end

# function for the summary statistics
function reduced_fourier_spectrum(u::Vector{Float64}, indices::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120)
  fourier_transform = abs.(fft(u))
  return fourier_transform[indices]
end

reduced_fourier_spectrum (generic function with 2 methods)

In [10]:
# FUNCTIONS NEEDED FOR SAVING THE RESULTS OF A SIMULATION

# function to create a new directory for each simulations, in order to store the needed files
function create_directory()
  base_path = pwd()
  base_path = joinpath(base_path, "Simulations")
  i = 1
  dir_name = "Real $i"
  dir_path = joinpath(base_path, dir_name)
  
  while isdir(dir_path)
    i += 1
    dir_name = "Real $i"
    dir_path = joinpath(base_path, dir_name)
  end
  
  mkpath(dir_path)
  println("Directory created at: $dir_path")
  cd(dir_path)
end

# function to save the prior as a string
function get_prior_string(prior)
  parts = []
  for d in prior.dists
    if isa(d, Uniform)
      push!(parts, "Uniform($(minimum(d)), $(maximum(d)))")
    else
      error("Unsupported distribution type: $(typeof(d))")
    end
  end
  
  return "product_distribution(" * join(parts, ", ") * ")"
end

# function to save the sabc parameters
function save_sabc_params(prior, n_particles::Int, n_simulation::Int, v::Float64, type::Int, indices::Union{Vector{Int}, StepRange{Int64, Int64}})
  curr_path = pwd()
  filename = "sabc_params.csv"
  path = joinpath(curr_path, filename)
    
  sabc_params = DataFrame(
    Parameter = ["prior", "n_particles", "n_simulation", "v", "type", "indices"],
    Value = [get_prior_string(prior), n_particles, n_simulation, v, type, string(indices)]
  )
    
 CSV.write(filename, sabc_params) 
 println("Parameters saved to: $path")
end

# Function to save the result object of a sABC algorithm
function save_result(result::SimulatedAnnealingABC.SABCresult{Vector{Float64}, Float64})
  curr_path = pwd()
  filenames = ["eps_hist.csv", "u_hist.csv", "rho_hist.csv"]
  variables = [result.state.ϵ_history, result.state.u_history, result.state.ρ_history]

  for (filename, variable) in zip(filenames, variables)
    labels = string.(1:size(variable, 1))
    path = joinpath(curr_path, filename)
    CSV.write(path, DataFrame(variable, labels))
    println("$filename data saved to: $path")
  end

  filename = "pop.csv"
  path = joinpath(curr_path, filename)

  param_samples = hcat(result.population...)

  posterior_params = DataFrame(
    N_value = param_samples[1, :],
    T_value = param_samples[2, :],
    tau_value = param_samples[3, :],
    sigma_value = param_samples[4, :],
    Bmax_value = param_samples[5, :]
  )

  CSV.write(path, posterior_params)
  println("Posterior parameters saved to: $path")

  filename = "rho.csv"
  path = joinpath(curr_path, filename)

  rho = result.ρ

  rho_values = DataFrame(rho, [:ss1, :ss2, :ss3, :ss4, :ss5, :ss6])

  CSV.write(path, rho_values)
  println("Rho values saved to: $path")
end

save_result (generic function with 1 method)

In [5]:
# THREADS SETTINGS AND INFO

ThreadPinning.pinthreads(:cores)
ThreadPinning.threadinfo()


System: 8 cores (no SMT), 8 sockets, 1 NUMA domains

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 

# = Julia thread, | = Socket seperator

Julia threads: 8
├ Occupied CPU-threads: 8
└ Mapping (Thread => CPUID): 1 => 0, 2 => 1, 3 => 2, 4 => 3, 5 => 4, ...


In [6]:
# EXTRACTING OPEN MAGNETIC FLUX AND SUNSPOT NUMBER RECORDS FROM XLSX FILE

# Define DataFrame object
data = DataFrame(
  year = Int[],
  open_magn_flux = Float64[],
  open_magn_flux_err = Float64[],
  ssa_open_magn_flux = Float64[],
  sunspots_num = Float64[],
  sunspots_err = Float64[],
  ssa_sunspots = Float64[]
)

# Open file and for each row write data into the DataFrame
XLSX.openxlsx("SN Usoskin Brehm.xlsx") do file
  sheet = file["Data"] 

  for row in XLSX.eachrow(sheet)
    if isa(row[2], Number)
      push!(data, (
        year = row[2],
        open_magn_flux = row[3],
        open_magn_flux_err = row[4],
        ssa_open_magn_flux = row[5],
        sunspots_num = row[7],
        sunspots_err = row[8],
        ssa_sunspots = row[9]
      ))
    end
  end
end

# Creation of the summary statistics for the real data
u = data.open_magn_flux
indices = [1, 2, 37, 50, 78, 85]

sim_ss = reduced_fourier_spectrum(u, indices)

6-element Vector{Float64}:
 3993.8700000000003
  649.4478744969342
  121.8823872718697
  128.89216890856196
  207.56096406106235
  274.8631648867697

In [7]:
# DIRECTORY MANAGING

# Current directory
initial_directory = pwd()

# New directory
create_directory()

# NB: After "create_directory", we move to the new directory.
# DON'T RUN THIS AGAIN, wait for the simulation to finish!!!! If you made errors,
# eliminate the Simulations/Simulation i directory and then rerun everything

Directory created at: /home/ubuntu/LCP_B/Project/Simulations/Real 10


In [8]:
# SIMULATION PARAMETERS MANAGING

# Parameters that can be tuned for new simulations
prior = product_distribution(Uniform(5, 15), Uniform(0.1, 10.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))
n_particles = 1000
n_simulation = 10000000
v = 1.0
type = 1

# Writing on file "sabc_params.csv" of the values set in this cell.
save_sabc_params(prior, n_particles, n_simulation, v, type, indices)

Parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/sabc_params.csv


In [9]:
# SIMULATION

# Conditions
tmin = data.year[1]; tmax = data.year[end]
Tsim = tmax

# Actual usage of the sABC algorithm
result = sabc(f_dist, prior;
              n_particles = n_particles, 
              n_simulation = n_simulation,
              v = v,
              type = type,
              indices = indices,
              fourier_data = sim_ss)

# Display of the summary of the results
display(result)

# Saving the results to the files: "eps_hist.csv", "u_hist.csv", "rho_hist.csv", "pop.csv", "rho.csv".
#save_result(result)

┌ Info: Preparing to run SABC algorithm: 'single-epsilon'
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:188
┌ Info: Using threads: 8 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:199
┌ Info: Set BLAS threads = 1 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:202
┌ Info: Set 'pinthreads(:cores)' for optimal multi-threading performance
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:204
┌ Info: Initializing population...
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:223
┌ Info: Initial resampling (δ = 0.1) - ESS = 996.6799495718246 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:277
┌ Info: Population

Approximate posterior sample with 1000 particles:
  - simulations used: 10000000
  - average transformed distance: 0.0002936
  - ϵ: [1.945e-5]
  - population resampling: 20
  - acceptance rate: 0.004007
The sample can be accessed with the field `population`.
The history of ϵ can be accessed with the field `state.ϵ_history`.
 -------------------------------------- 


In [11]:
save_result(result)

eps_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/eps_hist.csv
u_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/u_hist.csv
rho_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/rho_hist.csv
Posterior parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/pop.csv
Rho values saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 10/rho.csv


In [12]:
# DIRECTORY MANAGING

# WE go back to the initial directory
cd(initial_directory)
pwd()

"/home/ubuntu/LCP_B/Project"

In [13]:
rho = result.ρ

1000×6 Matrix{Float64}:
 0.0307596   0.00628604  0.033839    0.0241445   0.00541029  0.00969708
 0.0715326   0.0272761   0.0504851   0.00488449  0.00162995  0.00299215
 0.114309    0.0282498   0.0092395   0.0180508   0.00287073  0.0416147
 0.0166363   0.0322362   0.00418111  0.00410369  0.0200436   0.0381751
 0.0860073   0.0482326   0.0297286   0.0245134   0.0107785   0.0108465
 0.102923    0.0127828   0.0247135   0.00660343  0.0129281   0.00850982
 0.101616    0.0174523   0.00650522  0.0337245   0.019982    0.0117447
 0.0750283   0.0519299   0.00970323  0.012099    0.0184494   0.0163318
 0.0869253   0.0179406   0.0185448   0.00336167  0.0251784   0.00397843
 0.00560008  0.0521094   0.0606836   0.00517445  0.0353099   0.00237053
 ⋮                                                           ⋮
 0.0654594   0.0251954   0.00103182  0.0292956   0.0415554   0.00741373
 0.0920519   0.0163495   0.00342756  0.00114587  0.0125721   0.0152671
 0.0752547   0.0153096   0.0111028   0.00372208  0.0305

In [14]:
df = DataFrame(rho, :auto)

df_squared = DataFrame()

# Iterate over each column and compute the square of each element
for col in names(df)
    df_squared[!, col] = df[!, col] .^ 2
end

# Now `df_squared` contains the square of each entry in the original DataFrame
println(df_squared)

1000×6 DataFrame
  Row │ x1           x2           x3           x4           x5           x6          
      │ Float64      Float64      Float64      Float64      Float64      Float64     
──────┼──────────────────────────────────────────────────────────────────────────────
    1 │ 0.000946152  3.95143e-5   0.00114508   0.000582956  2.92713e-5   9.40333e-5
    2 │ 0.00511692   0.000743988  0.00254875   2.38582e-5   2.65674e-6   8.95293e-6
    3 │ 0.0130665    0.00079805   8.53683e-5   0.000325831  8.24106e-6   0.00173179
    4 │ 0.000276768  0.00103917   1.74817e-5   1.68403e-5   0.000401746  0.00145734
    5 │ 0.00739725   0.00232639   0.000883791  0.000600909  0.000116176  0.000117647
    6 │ 0.0105932    0.0001634    0.000610758  4.36053e-5   0.000167137  7.24171e-5
    7 │ 0.0103258    0.000304582  4.23179e-5   0.00113734   0.00039928   0.000137937
    8 │ 0.00562925   0.00269672   9.41527e-5   0.000146385  0.00034038   0.000266726
    9 │ 0.007556     0.000321866  0.00034391   1.1

In [15]:
row_sums = Vector{Float64}(undef, size(df, 1))

# Compute the sum of each row and store it in `row_sums`
for (i, row) in enumerate(eachrow(df_squared))
    row_sums[i] = sum(row)
end

# Now `row_sums` contains the sum of each row
k = 5  # Number of minimum values you want to find
new_indices = partialsortperm(row_sums, 1:k)  # Indices of the 5 smallest values
min_values = row_sums[new_indices]  # The 5 smallest values

println("Minimum values: ", min_values)
println("Indices of minimum values: ", new_indices)

Minimum values: [0.0006522692081695984, 0.0007671363720981072, 0.0011183741113196277, 0.0012340389299583703, 0.0012352157430594352]
Indices of minimum values: [440, 85, 126, 411, 128]


In [16]:
param_samples = hcat(result.population...)

posterior_params = DataFrame(
  N_value = param_samples[1, :],
  T_value = param_samples[2, :],
  tau_value = param_samples[3, :],
  sigma_value = param_samples[4, :],
  Bmax_value = param_samples[5, :]
)

Row,N_value,T_value,tau_value,sigma_value,Bmax_value
,Float64,Float64,Float64,Float64,Float64
1,5.22768,6.15935,1.80469,0.239198,7.98649
2,7.63451,8.01435,2.20293,0.263086,5.23695
3,9.68487,7.74219,2.86343,0.292976,5.41365
4,7.5972,7.87499,1.77054,0.287041,6.9945
5,5.13614,5.55797,1.90493,0.284965,7.9122
6,8.64356,6.69131,2.55661,0.246623,6.9989
7,9.09669,7.19629,2.76015,0.243066,6.61086
8,6.2049,5.29051,1.59196,0.29846,9.2036
9,6.45347,9.66678,1.0499,0.287725,8.05093


In [17]:
new_indices

5-element view(::Vector{Int64}, 1:5) with eltype Int64:
 440
  85
 126
 411
 128

In [18]:
df[new_indices, :]

Row,x1,x2,x3,x4,x5,x6
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0194808,0.0111734,0.00919226,0.00235995,0.000632054,0.00758013
2,0.0195001,0.000147369,0.000320841,0.0145271,0.00122656,0.0131991
3,0.00307507,0.021012,0.0120115,0.00627454,0.0204011,0.00821985
4,0.0209515,0.0170478,0.00898778,0.00648857,0.0195336,3.19189e-5
5,0.0242733,0.0128289,0.0160652,0.0131018,0.00715268,0.000728688


In [19]:
best_particles = posterior_params[new_indices, :]

Row,N_value,T_value,tau_value,sigma_value,Bmax_value
,Float64,Float64,Float64,Float64,Float64
1,8.09897,4.51895,3.63159,0.258871,7.81942
2,5.8251,6.90642,2.02787,0.285959,6.81222
3,9.20015,6.39844,2.94562,0.231808,5.83083
4,7.84125,3.76359,4.17398,0.290419,8.06362
5,8.62693,5.66445,3.20604,0.270327,7.15274
